# Strong authentication
A authentication system is known to be "weak" when an attacker who can obtain the password and replay it.

For instance, a typical password system is weak, because anyone who has access to the channel can view the password sent by the sender, and use the same credentials to authenticate with the receiver.

One example of strong authentication is the **challenge-response** system.

## Shared-key cryptography challenge-response
Suppose that both sender and receiver shares some secret key $k$.
Then the following process is performed for authentication
1. Alice requests communication with Bob
2. Bob randomly decides on a message $m$
3. Bob sends Alice the ciphertext $y = E_k(m)$
4. Alice decrypts the ciphertext and replies Bob with $m' = D_k(y)$
5. Bob verifies that $m'=m$, which is evidence that Alice knows the secret key $k$.

Notice that in this case, even if the attacker has full control over the channel, they cannot derive the secret key $k$ which was never sent through the channel directly.
And since $m$ is chosen randomly at the start of any communication, the attacker is also unable to use the $m$ or $m'$ they obtain to request a separate connection with the receiver.

## Public-key cryptography challenge-response
Similarly, we can perform the challenge-response in a public-key cryptography setting.

The following process is performed for authentication
1. Alice requests communication with Bob
2. Bob randomly decides on a number $r$ and sends it to Alice
3. Alice signs $r$ and sends her signature and her certificate to Bob
4. Bob verifies the authenticity of Alice's certificate, then obtain Alice's public key from it
5. Bob verifies that the signature on $r$ using her public key

In the above cases, $r$ is known as **nonces**, a number that is used only once during communication.

## Insufficiency of strong authentication
Suppose that the receiver has indeed authenticated that the one sending the message through the channel is a legitimate sender.
However, consider the case when an attacker is able to hijack the channel at any point in time.
This means that they can impersonate as the sender after the authentication by the receiver, thus nullifying the authentication performed at the start.

Hence, strong authentication system, on its own, is unable to the secure the channel.
Therefore, to secure the channel in an event that the attacker can compromise the channel after authentication, the communicating parties must share some form of secret key.

# Key Exchange
For now, we assume that the attacker can only sniff the traffic on the channel, and not able to modify the traffic.
We wish to device a scheme where both sender and receiver can agree on some secret using this insecure channel.

## Unauthenticated PKC-based key exchange
With PKC, the process is straight forward:
1. Alice generates a public-private key pair
2. Alice sends her public key to Bob
3. Bob randomly decides on a session key $k$
4. Bob encrypts $k$ with Alice's public key, and sends it over to Alice
5. Alice decrypts $k$ using her private key

With this, the attacker can only see the encrypted session key on the channel.
Since the attacker is unable to derive the session key without Alice's private key, the session is only known the Alice and Bob at the end of the exchange, as desired.

## Unauthenticated Diffie-Hellman key exchange (DH)
We pre-require Alice and Bob to agree on a generator $g$ and a large prime $p$.
1. Alice randomly decides on $a$ and computes $x = g^a \mod p$
2. At the same time, Bob randomly decides on $b$ and computes $y = g^b \mod p$
3. Alice sends $x$ to Bob, while Bob sends $y$ to Alice
4. Alice computes $k_a = y^a = g^{ba}$ while Bob computes $k_b= x^b = g^{ab}$.
In any case, $k_a = k_b$, which is the session key that they now agree on.

A simplified analogy is:
1. Both Alice and Bob shares some secret number $s$
2. Alice generates $a$ and Bob generates $b$
3. Alice sends $x=a+s$ to Bob and Bob sends $y=b+s$ to Alice
4. Alice computes $k_a=y+a$ and Bob computes $k_b=x+b$

Since the attacker do not know $a,b$ or $s$, they are unable to compute the session key $k_a,k_b$ using only $x$ and $y$ which they can see on the channel.

---
Bringing it back, note that we assumed that the attacker is unable to modify the traffic on the channel.
Consider the case if the attacker is able to hijack the channel.

The attacker can perform the following **man-in-the-middle attack**:
1. Wait until the sender wishes to communicate with the receiver
2. Intercepts the request
    1. Respond to the sender's request and perform the key exchange
    2. Initiate communication with the receiver via key exchange
3. With the previous step, the attacker now shares a session key with the sender $k_1$, and a session key with the receiver $k_2$.
4. When the sender wishes to send a message, the attacker will intercept it and decrypt it using $k_1$
5. The attacker can now forward (or modify) the message from the sender to the receiver, by encrypting it using $k_2$

This arose because key exchange (on its own) only guarantees confidentiality, where no third party can view the message sent on the channel.
It fails to guarantee authenticity, wish ensures that the entity that we are communicating with is verified.
Hence, the solution is clear, simply incorporate authentication to the key exchange protocols.

## Station-to-station protocol
This protocol adds signatures to the DH.
We pre-require that one party knows the other's public key.

1. Alice randomly decides on $a$ and computes $x = g^a \mod p$
2. At the same time, Bob randomly decides on $b$ and computes $y = g^b \mod p$
3. **Bob signs his $y$ to derive $s$**
3. Alice sends $x$ to Bob, while Bob sends $(y,s)$ to Alice
4. Alice computes $k_a = y^a = g^{ba}$ while Bob computes $k_b= x^b = g^{ab}$.
5. **Alice verifies that the signature is valid, and thus the entity is indeed Bob**

This forms the basis for unilateral authentication.
We can easily extend it to bilateral authentication if we wish so.

Usually, a set of session keys are generated and shared during key exchange.
For example, a key will be used to encrypt the messages sent, while another will be used to produce [MAC](./data_origin_authentication.ipynb#mac) to ensure that the messages are not modified.

# Secure Channel
A **secure channel** guarantees **confidentiality, integrity and authenticity** against all attackers who can hijack the network.

To ensure confidentiality, we require encryption; while for integrity/authenticity, we use MAC/digital signature.

Thus, putting it all together, to obtain a secure channel, the following is to be done.
(Once again, we assume unilateral communication for ease of discussion.
The following can be easily extended to bilateral communication)

1. Sender and receiver perform unilateral authenticated key exchange, obtaining session key of $(k, t)$.
2. Using the authentication, sender can be assured of the identity of the receiver.
3. Suppose that the sender has a series of messages to send, $m_1, m_2, m_3 \dots$.
4. The sender will then send $(E_k(i || m_i), MAC_t(i || m_i))$, where $||$ is the concatenation operator.

Note that we perform "encrypt-then-MAC", there are other schemes such as "MAC-then-encrypt" or "MAC-and-encrypt", each with their own characteristics.

Notice that we appended the sequence number $i$ to the message to be encrypted.
This is to prevent replay attacks by the attacker.
Consider the case where Alice sends Bob the instruction of "Give me $100".
Without the sequence number, the attacker can capture Alice's traffic, and repeat the same instruction over to Bob using the same session.
Thus, the attacker can falsely trick Bob into draining his account.

The sequence number prevents the same message to be illegitimately replayed over the channel.

## Secure channels in the wild
HTTPS is used to secure web traffic.
It is built on top of SSL/TLS, which are protocols to secure communication.

TLS in a nutshell:
1. Cipher negotiation
2. Authenticated key exchange
3. Symmetric-key based secure communication
4. Renegotiate key if need be